In [1]:
!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 9.9 MB/s eta 0:00:00
  Created wheel for ctransformers: filename=ctransformers-0.2.27-cp310-cp310-linux_x86_64.whl size=2338882 sha256=0e792a7dbbd5d4849efdd275a9cd2411e979511bcb92844366707df4ac48e23e
  Stored in directory: /root/.cache/pip/wheels/dd/54/e9/32364da8eee84a2b0b412394983c15add18816c507e90f02d8
Successfully built ctransformers


In [2]:
from ctransformers import AutoModelForCausalLM

model_id = "TheBloke/Llama-2-7B-chat-GGML"

config = {'max_new_tokens': 100, 'repetition_penalty': 1.1,
          'temperature': 0.01, 'stream': True}

llm = AutoModelForCausalLM.from_pretrained(
      model_id,
      model_type="llama",
      #lib='avx2', for cpu use
      gpu_layers=110,
      **config
      )

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

(…)389e1d4b91cab2cf470aab11864b/config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-7b-chat.ggmlv3.q2_K.bin:   0%|          | 0.00/2.87G [00:00<?, ?B/s]

In [ ]:
!unzip "/content/First_Phase_Text_Dataset.zip" -d "/content/aicup"

In [10]:
import pandas as pd
#設定一個list
set1_df = pd.read_csv("/content/aicup/Tutorial/opendid_set1.tsv",sep='\t',header=None,names=['file_name','start','content','PHI'])
ans_df = pd.read_csv("/content/aicup/Tutorial/opendid_valid.tsv",sep='\t',header=None,names=['file_name','start','content'])
print(set1_df)
print(ans_df)

      file_name  start                  content  \
0            10      1  Episode No:  09F016547J   
1            10     25               091016.NMT   
2            10     37            SIZAR, HOWARD   
3            10     52        Lab No:  09F01654   
4            10     70                  Runford   
...         ...    ...                      ...   
85731  file9965  18363   I Luague and I Eifert.   
85732  file9965  18391     SpecimenReceivedDate   
85733  file9965  18412      2512-10-20 00:00:00   
85734  file9965  18434                 LastName   
85735  file9965  18443                   Bodway   

                                                  PHI  
0                                   IDNUM: 09F016547J  
1                           MEDICALRECORD: 091016.NMT  
2                              PATIENT: SIZAR, HOWARD  
3                                     IDNUM: 09F01654  
4                                     STREET: Runford  
...                                               .

In [19]:
print(len(set1_df))
print(set1_df.iloc[1, 2])

85736
091016.NMT


In [ ]:
for row in range(0,len(set1_df)):
  example_input=set1_df.iloc[row, 2]
  example_output=set1_df.iloc[row, 3]
  input="""Episode No:  13R065915L
  9305066.RAN

  SMALLWOOD, KERRIE
  Lab No:  13H01081
  40 QUINN STREET
  """

  # pt-br for multilingual eval
  prompt_template= f"""You are now an expert for deidentification task. I will give you some input, your job is to list the protected health information (PHI).
  Here is one example:
  Input:
  {example_input}
  Output:
  {example_output}
  """
  llm(prompt_template, stream=False)

In [ ]:
ans_phi = []
for row in range(0,len(ans_df)):
  ans_input=ans_df.iloc[row, 2]
  prompt_template= f"""
  Input:
  {ans_input}
  Output:"""
  Output = llm(prompt_template, stream=False)
  print(Output)
  ans_phi.append(Output)

In [ ]:
ans_df.loc[len(ans_df.index)] = ans_phi